In [1]:
from time import sleep
import json
from kafka import KafkaProducer
from kafka import KafkaConsumer

In [ ]:
#channel
topic='app'

# producer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda x: json.dumps(x).encode('utf-8'))

#consumer
consumer = KafkaConsumer(topic, bootstrap_servers=['localhost:9092'],auto_offset_reset='earliest',value_deserializer=lambda x:json.loads(x.decode('utf-8')))


In [ ]:
for e in range(5):
    data = {'number' : e}
    producer.send(topic, value=data)
    sleep(1)

In [ ]:
for message in consumer:
    message = message.value
    print('Message :{}'.format(message))

In [2]:
import tweepy
import os

In [3]:
# twitter setup
consumer_key = os.getenv('CONSUMERTOKEN')
consumer_secret = os.getenv('CONSUMERSECRET')
access_token = os.getenv('TOKEN')
access_token_secret = os.getenv('SECRETTOKEN')
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object by passing in auth information
api = tweepy.API(auth) 

In [4]:
from datetime import datetime, timedelta

def normalize_timestamp(time):
    mytime = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    mytime += timedelta(hours=1)   # the tweets are timestamped in GMT timezone, while I am in +1 timezone
    return (mytime.strftime("%Y-%m-%d %H:%M:%S")) 

In [5]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')
topic_name = 'tweets-lambda1'

In [6]:
def get_twitter_data():
    res = api.search("Apple OR iphone OR iPhone")
    for i in res:
        record = ''
        record += str(i.user.id_str)
        record += ';'
        record += str(normalize_timestamp(str(i.created_at)))
        record += ';'
        record += str(i.user.followers_count)
        record += ';'
        record += str(i.user.location)
        record += ';'
        record += str(i.favorite_count)
        record += ';'
        record += str(i.retweet_count)
        record += ';'
        producer.send(topic_name, str.encode(record))

In [7]:
get_twitter_data()

AttributeError: 'API' object has no attribute 'search'